# Практическое задание к уроку 5

In [1]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score,roc_auc_score
from sklearn.ensemble import RandomForestRegressor
import scipy.stats as st
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score

import time
warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
lst_data={'train': pd.read_csv('data\\train.csv'),
          'test': pd.read_csv('data\\test.csv'),
          'payment' : pd.read_csv('data\\payments.csv'),
          'client profile': pd.read_csv('data\\client_profile.csv'),
          'bki': pd.read_csv('data\\bki.csv'),
          'apl_history': pd.read_csv('data\\applications_history.csv')          
         }

In [3]:
for name, el in lst_data.items():
    print(name, el.shape)
    print(el['APPLICATION_NUMBER'].value_counts().max())

train (110093, 3)
1
test (165141, 2)
1
payment (1023932, 8)
37
client profile (250000, 24)
1
bki (945234, 17)
63
apl_history (1670214, 26)
77


In [4]:
data = pd.concat([ lst_data['train'],lst_data['test']], axis=0)
data = data.reset_index(drop=True)

In [5]:
data.shape

(275234, 3)

#### Client profile

In [6]:
lst_data['client profile'].describe()

,APPLICATION_NUMBER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,...,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,2.500000e+05,250000.000000,2.500000e+05,2.500000e+05,249989.000000,250000.000000,250000.000000,250000.000000,85041.000000,250000.000000,...,249998.000000,109121.000000,2.494680e+05,200462.000000,216293.000000,216293.000000,216293.000000,216293.000000,216293.000000,216293.000000
mean,1.235771e+08,0.417624,1.690662e+05,5.993051e+05,27125.209585,0.020871,16033.460496,67505.883296,12.030221,0.280244,...,2.154025,0.502122,5.141749e-01,0.510848,0.006348,0.006866,0.034217,0.267475,0.266786,1.899030
std,8.877722e+04,0.722529,2.588480e+05,4.022720e+05,14503.930366,0.013827,4361.332320,139261.099609,11.899404,0.449119,...,0.911543,0.210817,1.910938e-01,0.195054,0.083783,0.107957,0.204029,0.914514,0.831547,1.868811
min,1.234233e+08,0.000000,2.565000e+04,4.500000e+04,1615.500000,0.000290,7489.000000,0.000000,0.000000,0.000000,...,1.000000,0.014691,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.235002e+08,0.000000,1.125000e+05,2.700000e+05,16524.000000,0.010006,12415.000000,931.000000,5.000000,0.000000,...,2.000000,0.334180,3.920026e-01,0.370650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.235772e+08,0.000000,1.485000e+05,5.147775e+05,24930.000000,0.018850,15744.000000,2217.000000,9.000000,0.000000,...,2.000000,0.506018,5.656713e-01,0.535276,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.236540e+08,1.000000,2.025000e+05,8.086500e+05,34618.500000,0.028663,19681.000000,5678.000000,15.000000,1.000000,...,3.000000,0.674793,6.635195e-01,0.669057,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.237309e+08,19.000000,1.170000e+08,4.050000e+06,230161.500000,0.072508,25229.000000,365243.000000,69.000000,1.000000,...,20.000000,0.962693,8.549997e-01,0.896010,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [7]:
profile=lst_data['client profile']
profile.columns = [col.lower() for col in profile.columns]

In [8]:
data.columns = [col.lower() for col in data.columns]

In [9]:
def create_client_profile_features(X: pd.DataFrame) -> pd.DataFrame:
   
    X = X.copy()

    X["days_on_last_job"] = X["days_on_last_job"].replace(365243, np.nan)
    bki_flags = [flag for flag in X.columns if "amt_req_credit_bureau" in flag]
    X["bki_requests_count"] = X[bki_flags].sum(axis=1)
  

    X["external_scoring_prod"] = X["external_scoring_rating_1"] * X["external_scoring_rating_2"] * X["external_scoring_rating_3"]
    X["external_scoring_weighted"] = X.external_scoring_rating_1 * 2 + X.external_scoring_rating_2 * 1 + X.external_scoring_rating_3 * 3

    for function_name in ["min", "max", "mean", "nanmedian", "var"]:
        feature_name = "external_scoring_rating_{}".format(function_name)
        X[feature_name] = eval("np.{}".format(function_name))(
            X[["external_scoring_rating_1", "external_scoring_rating_2", "external_scoring_rating_3"]], axis=1
        )

    # Отношение между основными фин. показателями
    X['ratio_credit_to_annuity'] = X['amount_credit'] / X['amount_annuity']
    X["ratio_annuity_to_salary"] = X['amount_annuity'] / X['total_salary']
    X['ratio_credit_to_salary'] = X['amount_credit'] / X['total_salary']
    #X["total_salary_net"] = X["total_salary"] - X["amount_annuity"]

    # Отношение фин. показателей к возрасту и временным фичам
    X["ratio_annuity_to_age"] = X["amount_annuity"] / X["age"]
    X["ratio_credit_to_age"] = X["amount_credit"] / X["age"]
    X["ratio_salary_to_age"] = X["total_salary"] / X["age"]
    X["ratio_salary_to_experience"] = X["total_salary"] / X["days_on_last_job"]
    X["ratio_credit_to_experience"] = X["amount_credit"] / X["days_on_last_job"]
    X["ratio_annuity_to_experience"] = X["amount_annuity"] / X["days_on_last_job"]

    # Отношение врменных признаков
    X["ratio_age_to_experience"] = X["age"] / X["days_on_last_job"]
    X["ratio_salary_to_region_population"] = X["total_salary"] * X["region_population"]
    X["ratio_car_to_experience"] = X["own_car_age"] / X["days_on_last_job"]
    X["ratio_car_to_age"] = X["own_car_age"] / X["age"]

    # Произведение фин. показателей кредита на вероятность дефолта
    # Такая штука называется математическим ожиданием дефолта или ожидаемыми потерями
    X["expected_total_loss_1"] = X["external_scoring_rating_1"] * X["amount_credit"]
    X["expected_total_loss_2"] = X["external_scoring_rating_2"] * X["amount_credit"]
    X["expected_total_loss_3"] = X["external_scoring_rating_3"] * X["amount_credit"]
    X["expected_monthly_loss_1"] = X["external_scoring_rating_1"] * X["amount_annuity"]
    X["expected_monthly_loss_2"] = X["external_scoring_rating_2"] * X["amount_annuity"]
    X["expected_monthly_loss_3"] = X["external_scoring_rating_3"] * X["amount_annuity"]

    return X

In [10]:
data = data.merge(
    create_client_profile_features(lst_data['client profile']), how="left", on="application_number"
)

In [11]:
mask = data["target"].isnull()
features_to_drop = ["application_number", "target"]

train, test = data.loc[~mask], data.loc[mask]

target, test_id = train["target"], test["application_number"]
train = train.drop(features_to_drop, axis=1)
test = test.drop(features_to_drop, axis=1)

categorial = train.dtypes[train.dtypes == "object"].index
numerical = list(set(train.columns) - set(categorial))

train = train.replace(np.inf, np.nan)
train = train.replace(-np.inf, np.nan)

In [12]:
train

,name_contract_type,gender,childrens,total_salary,amount_credit,amount_annuity,education_level,family_status,region_population,age,...,ratio_age_to_experience,ratio_salary_to_region_population,ratio_car_to_experience,ratio_car_to_age,expected_total_loss_1,expected_total_loss_2,expected_total_loss_3,expected_monthly_loss_1,expected_monthly_loss_2,expected_monthly_loss_3
0,Cash,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728.0,...,9.149506,3008.4075,0.006399,0.000699,599170.547652,552256.266546,612667.559305,17609.307043,16230.521013,18005.977111
1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cash,F,0.0,135000.0,1006920.0,42660.0,Higher education,Married,0.026392,21557.0,...,5.958264,3562.9200,NaN,NaN,NaN,686869.876357,269722.588880,NaN,29100.493510,11427.288803
3,Cash,M,0.0,180000.0,518562.0,22972.5,Secondary / secondary special,Married,0.031329,22338.0,...,NaN,5639.2200,NaN,NaN,NaN,88829.188848,88386.882459,NaN,3935.167908,3915.573562
4,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110088,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110089,Cash,F,0.0,175500.0,269550.0,12618.0,Secondary / secondary special,Married,0.010032,10519.0,...,7.768833,1760.6160,NaN,NaN,142348.303154,127166.558499,156738.932900,6663.516562,5952.838565,7337.161400
110090,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110091,Cash,F,2.0,270000.0,1024740.0,49428.0,Incomplete higher,Married,0.019101,15008.0,...,41.573407,5157.2700,0.060942,0.001466,670001.539719,667338.205604,572324.081646,32317.305956,32188.840903,27605.865593


#### 1. Обучить алгоритмы LightGBM и XGBoost, получить OOF прогнозы, оценить корреляцию прогнозов на обучающей выборке. Применить модели на тестовую выборку и оценить корреляцию.

In [13]:
categorial = train.dtypes[train.dtypes == "object"].index
numerical = list(set(train.columns) - set(categorial))

In [14]:
x_train, x_valid = train_test_split(
      train, train_size=0.7, random_state=27
    )
y_train, y_valid = train_test_split(
        target, train_size=0.7, random_state=27
    )

cb_params = {
    "n_estimators": 2000,
    "learning_rate": 0.01,
    "loss_function": "Logloss",
    "eval_metric": "auc",
 #   "task_type": "CPU",
    "max_bin": 20,
    "verbose": 1000,
    "max_depth": 6,
    "l2_leaf_reg": 10,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
   
}
model_lgbm = lgb.LGBMClassifier(**cb_params)
       
model_lgbm.fit(x_train[numerical], y_train,eval_set=[(x_train[numerical], y_train), (x_valid[numerical], y_valid)])
oof_preds_lgbm= model_lgbm.predict_proba(x_valid[numerical])[:, 1]
oof_preds_train_lgbm=model_lgbm.predict_proba(x_train[numerical])[:, 1]


[1]	valid_0's binary_logloss: 0.279508	valid_1's binary_logloss: 0.281865
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.278976	valid_1's binary_logloss: 0.281385
[3]	valid_0's binary_logloss: 0.278463	valid_1's binary_logloss: 0.280913
[4]	valid_0's binary_logloss: 0.277961	valid_1's binary_logloss: 0.280459
[5]	valid_0's binary_logloss: 0.277478	valid_1's binary_logloss: 0.280018
[6]	valid_0's binary_logloss: 0.277009	valid_1's binary_logloss: 0.27959
[7]	valid_0's binary_logloss: 0.276555	valid_1's binary_logloss: 0.27918
[8]	valid_0's binary_logloss: 0.276112	valid_1's binary_logloss: 0.278769
[9]	valid_0's binary_logloss: 0.275684	valid_1's binary_logloss: 0.278386
[10]	valid_0's binary_logloss: 0.275272	valid_1's binary_logloss: 0.278008
[11]	valid_0's binary_logloss: 0.274862	valid_1's binary_logloss: 0.277634
[12]	valid_0's binary_logloss: 0.274463	valid_1's binary_logloss: 0.277287
[13]	valid_0's binary_logloss: 0.274081	valid_1's 

[109]	valid_0's binary_logloss: 0.257056	valid_1's binary_logloss: 0.26228
[110]	valid_0's binary_logloss: 0.256979	valid_1's binary_logloss: 0.262214
[111]	valid_0's binary_logloss: 0.256899	valid_1's binary_logloss: 0.262159
[112]	valid_0's binary_logloss: 0.256821	valid_1's binary_logloss: 0.2621
[113]	valid_0's binary_logloss: 0.256743	valid_1's binary_logloss: 0.262042
[114]	valid_0's binary_logloss: 0.256662	valid_1's binary_logloss: 0.261976
[115]	valid_0's binary_logloss: 0.256585	valid_1's binary_logloss: 0.261917
[116]	valid_0's binary_logloss: 0.256504	valid_1's binary_logloss: 0.261858
[117]	valid_0's binary_logloss: 0.256426	valid_1's binary_logloss: 0.261799
[118]	valid_0's binary_logloss: 0.256348	valid_1's binary_logloss: 0.261737
[119]	valid_0's binary_logloss: 0.256276	valid_1's binary_logloss: 0.261681
[120]	valid_0's binary_logloss: 0.2562	valid_1's binary_logloss: 0.261626
[121]	valid_0's binary_logloss: 0.256126	valid_1's binary_logloss: 0.261577
[122]	valid_0's b

[217]	valid_0's binary_logloss: 0.251131	valid_1's binary_logloss: 0.258672
[218]	valid_0's binary_logloss: 0.251092	valid_1's binary_logloss: 0.258663
[219]	valid_0's binary_logloss: 0.251051	valid_1's binary_logloss: 0.258638
[220]	valid_0's binary_logloss: 0.251012	valid_1's binary_logloss: 0.258616
[221]	valid_0's binary_logloss: 0.250973	valid_1's binary_logloss: 0.258603
[222]	valid_0's binary_logloss: 0.250925	valid_1's binary_logloss: 0.258574
[223]	valid_0's binary_logloss: 0.250887	valid_1's binary_logloss: 0.258553
[224]	valid_0's binary_logloss: 0.250844	valid_1's binary_logloss: 0.258529
[225]	valid_0's binary_logloss: 0.250798	valid_1's binary_logloss: 0.258501
[226]	valid_0's binary_logloss: 0.250758	valid_1's binary_logloss: 0.258482
[227]	valid_0's binary_logloss: 0.250711	valid_1's binary_logloss: 0.258452
[228]	valid_0's binary_logloss: 0.250675	valid_1's binary_logloss: 0.258442
[229]	valid_0's binary_logloss: 0.250629	valid_1's binary_logloss: 0.258411
[230]	valid_

[325]	valid_0's binary_logloss: 0.247156	valid_1's binary_logloss: 0.257273
[326]	valid_0's binary_logloss: 0.247125	valid_1's binary_logloss: 0.25727
[327]	valid_0's binary_logloss: 0.24709	valid_1's binary_logloss: 0.25726
[328]	valid_0's binary_logloss: 0.24706	valid_1's binary_logloss: 0.257252
[329]	valid_0's binary_logloss: 0.247033	valid_1's binary_logloss: 0.257246
[330]	valid_0's binary_logloss: 0.247002	valid_1's binary_logloss: 0.257241
[331]	valid_0's binary_logloss: 0.246969	valid_1's binary_logloss: 0.257239
[332]	valid_0's binary_logloss: 0.246942	valid_1's binary_logloss: 0.257232
[333]	valid_0's binary_logloss: 0.246904	valid_1's binary_logloss: 0.257225
[334]	valid_0's binary_logloss: 0.246874	valid_1's binary_logloss: 0.257222
[335]	valid_0's binary_logloss: 0.246847	valid_1's binary_logloss: 0.257215
[336]	valid_0's binary_logloss: 0.246817	valid_1's binary_logloss: 0.257212
[337]	valid_0's binary_logloss: 0.246791	valid_1's binary_logloss: 0.257204
[338]	valid_0's 

[433]	valid_0's binary_logloss: 0.244021	valid_1's binary_logloss: 0.256796
[434]	valid_0's binary_logloss: 0.243998	valid_1's binary_logloss: 0.256792
[435]	valid_0's binary_logloss: 0.243973	valid_1's binary_logloss: 0.256796
[436]	valid_0's binary_logloss: 0.243946	valid_1's binary_logloss: 0.256796
[437]	valid_0's binary_logloss: 0.243923	valid_1's binary_logloss: 0.256796
[438]	valid_0's binary_logloss: 0.243894	valid_1's binary_logloss: 0.256797
[439]	valid_0's binary_logloss: 0.24387	valid_1's binary_logloss: 0.256798
[440]	valid_0's binary_logloss: 0.243845	valid_1's binary_logloss: 0.256796
[441]	valid_0's binary_logloss: 0.243824	valid_1's binary_logloss: 0.256796
[442]	valid_0's binary_logloss: 0.243797	valid_1's binary_logloss: 0.256791
[443]	valid_0's binary_logloss: 0.243767	valid_1's binary_logloss: 0.256787
[444]	valid_0's binary_logloss: 0.243745	valid_1's binary_logloss: 0.256788
[445]	valid_0's binary_logloss: 0.243716	valid_1's binary_logloss: 0.256784
[446]	valid_0

[541]	valid_0's binary_logloss: 0.241599	valid_1's binary_logloss: 0.256689
[542]	valid_0's binary_logloss: 0.241568	valid_1's binary_logloss: 0.256686
[543]	valid_0's binary_logloss: 0.241542	valid_1's binary_logloss: 0.256684
[544]	valid_0's binary_logloss: 0.241514	valid_1's binary_logloss: 0.256688
[545]	valid_0's binary_logloss: 0.241508	valid_1's binary_logloss: 0.256687
[546]	valid_0's binary_logloss: 0.241479	valid_1's binary_logloss: 0.256685
[547]	valid_0's binary_logloss: 0.241447	valid_1's binary_logloss: 0.256684
[548]	valid_0's binary_logloss: 0.24142	valid_1's binary_logloss: 0.256681
[549]	valid_0's binary_logloss: 0.24139	valid_1's binary_logloss: 0.25668
[550]	valid_0's binary_logloss: 0.241369	valid_1's binary_logloss: 0.256679
[551]	valid_0's binary_logloss: 0.241345	valid_1's binary_logloss: 0.256679
[552]	valid_0's binary_logloss: 0.24132	valid_1's binary_logloss: 0.256675
[553]	valid_0's binary_logloss: 0.241314	valid_1's binary_logloss: 0.256674
[554]	valid_0's 

In [15]:
score = roc_auc_score(y_valid, oof_preds_lgbm)
score

0.7246331856414824

In [16]:
cb_params = {
    "n_estimators": 2000,
    "learning_rate": 0.01,
    "loss_function": "Logloss",
    "eval_metric": "auc",
 #   "task_type": "CPU",
    "max_bin": 20,
    "verbose": 1000,
    "max_depth": 6,
    "l2_leaf_reg": 10,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
   
}
model_xgb = xgb.XGBClassifier(**cb_params)
       
model_xgb.fit(x_train[numerical], y_train,eval_set=[(x_train[numerical], y_train), (x_valid[numerical], y_valid)])
oof_preds_xgb= model_xgb.predict_proba(x_valid[numerical])[:, 1]
oof_preds_train_xgb=model_xgb.predict_proba(x_train[numerical])[:, 1]

[09:24:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { early_stopping_rounds, l2_leaf_reg, loss_function, random_seed, thread_count, verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.70395	validation_1-auc:0.69987
[1]	validation_0-auc:0.70556	validation_1-auc:0.70155
[2]	validation_0-auc:0.70561	validation_1-auc:0.70210
[3]	validation_0-auc:0.70630	validation_1-auc:0.70165
[4]	validation_0-auc:0.70708	validation_1-auc:0.70217
[5]	validation_0-auc:0.70719	validation_1-auc:0.70218
[6]	validation_0-auc:0.70825	validation_1-auc:0.70250
[7]	validation_0-auc:0.70847	validation_1-auc:0.70247
[8]	validation_0-auc:0.70947	validation_1-auc:0.70302
[9]	validation_0-auc:0.70957	validation_1-auc:0.703

[140]	validation_0-auc:0.73852	validation_1-auc:0.71167
[141]	validation_0-auc:0.73869	validation_1-auc:0.71151
[142]	validation_0-auc:0.73874	validation_1-auc:0.71145
[143]	validation_0-auc:0.73880	validation_1-auc:0.71139
[144]	validation_0-auc:0.73892	validation_1-auc:0.71140
[145]	validation_0-auc:0.73910	validation_1-auc:0.71142
[146]	validation_0-auc:0.73915	validation_1-auc:0.71141
[147]	validation_0-auc:0.73928	validation_1-auc:0.71155
[148]	validation_0-auc:0.73929	validation_1-auc:0.71166
[149]	validation_0-auc:0.73937	validation_1-auc:0.71166
[150]	validation_0-auc:0.73941	validation_1-auc:0.71170
[151]	validation_0-auc:0.73955	validation_1-auc:0.71170
[152]	validation_0-auc:0.73962	validation_1-auc:0.71166
[153]	validation_0-auc:0.73983	validation_1-auc:0.71181
[154]	validation_0-auc:0.73987	validation_1-auc:0.71178
[155]	validation_0-auc:0.74000	validation_1-auc:0.71208
[156]	validation_0-auc:0.74019	validation_1-auc:0.71200
[157]	validation_0-auc:0.74030	validation_1-auc:

[287]	validation_0-auc:0.75904	validation_1-auc:0.71749
[288]	validation_0-auc:0.75926	validation_1-auc:0.71744
[289]	validation_0-auc:0.75938	validation_1-auc:0.71758
[290]	validation_0-auc:0.75951	validation_1-auc:0.71759
[291]	validation_0-auc:0.75961	validation_1-auc:0.71757
[292]	validation_0-auc:0.75973	validation_1-auc:0.71752
[293]	validation_0-auc:0.75995	validation_1-auc:0.71762
[294]	validation_0-auc:0.76011	validation_1-auc:0.71771
[295]	validation_0-auc:0.76021	validation_1-auc:0.71776
[296]	validation_0-auc:0.76036	validation_1-auc:0.71766
[297]	validation_0-auc:0.76055	validation_1-auc:0.71770
[298]	validation_0-auc:0.76068	validation_1-auc:0.71777
[299]	validation_0-auc:0.76088	validation_1-auc:0.71775
[300]	validation_0-auc:0.76097	validation_1-auc:0.71783
[301]	validation_0-auc:0.76114	validation_1-auc:0.71772
[302]	validation_0-auc:0.76123	validation_1-auc:0.71767
[303]	validation_0-auc:0.76135	validation_1-auc:0.71769
[304]	validation_0-auc:0.76156	validation_1-auc:

[434]	validation_0-auc:0.78229	validation_1-auc:0.72312
[435]	validation_0-auc:0.78242	validation_1-auc:0.72315
[436]	validation_0-auc:0.78256	validation_1-auc:0.72319
[437]	validation_0-auc:0.78265	validation_1-auc:0.72319
[438]	validation_0-auc:0.78279	validation_1-auc:0.72324
[439]	validation_0-auc:0.78298	validation_1-auc:0.72336
[440]	validation_0-auc:0.78311	validation_1-auc:0.72337
[441]	validation_0-auc:0.78317	validation_1-auc:0.72343
[442]	validation_0-auc:0.78325	validation_1-auc:0.72339
[443]	validation_0-auc:0.78352	validation_1-auc:0.72340
[444]	validation_0-auc:0.78362	validation_1-auc:0.72343
[445]	validation_0-auc:0.78375	validation_1-auc:0.72350
[446]	validation_0-auc:0.78391	validation_1-auc:0.72359
[447]	validation_0-auc:0.78391	validation_1-auc:0.72361
[448]	validation_0-auc:0.78403	validation_1-auc:0.72364
[449]	validation_0-auc:0.78422	validation_1-auc:0.72365
[450]	validation_0-auc:0.78447	validation_1-auc:0.72380
[451]	validation_0-auc:0.78463	validation_1-auc:

[581]	validation_0-auc:0.79790	validation_1-auc:0.72723
[582]	validation_0-auc:0.79789	validation_1-auc:0.72730
[583]	validation_0-auc:0.79795	validation_1-auc:0.72730
[584]	validation_0-auc:0.79804	validation_1-auc:0.72740
[585]	validation_0-auc:0.79805	validation_1-auc:0.72740
[586]	validation_0-auc:0.79814	validation_1-auc:0.72747
[587]	validation_0-auc:0.79825	validation_1-auc:0.72753
[588]	validation_0-auc:0.79827	validation_1-auc:0.72747
[589]	validation_0-auc:0.79834	validation_1-auc:0.72742
[590]	validation_0-auc:0.79835	validation_1-auc:0.72743
[591]	validation_0-auc:0.79841	validation_1-auc:0.72743
[592]	validation_0-auc:0.79845	validation_1-auc:0.72745
[593]	validation_0-auc:0.79848	validation_1-auc:0.72745
[594]	validation_0-auc:0.79860	validation_1-auc:0.72762
[595]	validation_0-auc:0.79866	validation_1-auc:0.72757
[596]	validation_0-auc:0.79874	validation_1-auc:0.72759
[597]	validation_0-auc:0.79881	validation_1-auc:0.72765
[598]	validation_0-auc:0.79884	validation_1-auc:

...

[1309]	validation_0-auc:0.83710	validation_1-auc:0.73017
[1310]	validation_0-auc:0.83712	validation_1-auc:0.73018
[1311]	validation_0-auc:0.83715	validation_1-auc:0.73017
[1312]	validation_0-auc:0.83716	validation_1-auc:0.73016
[1313]	validation_0-auc:0.83733	validation_1-auc:0.73017
[1314]	validation_0-auc:0.83734	validation_1-auc:0.73017
[1315]	validation_0-auc:0.83736	validation_1-auc:0.73010
[1316]	validation_0-auc:0.83737	validation_1-auc:0.73009
[1317]	validation_0-auc:0.83739	validation_1-auc:0.73008
[1318]	validation_0-auc:0.83740	validation_1-auc:0.73007
[1319]	validation_0-auc:0.83742	validation_1-auc:0.73008
[1320]	validation_0-auc:0.83743	validation_1-auc:0.73001
[1321]	validation_0-auc:0.83745	validation_1-auc:0.73002
[1322]	validation_0-auc:0.83746	validation_1-auc:0.73002
[1323]	validation_0-auc:0.83749	validation_1-auc:0.73002
[1324]	validation_0-auc:0.83749	validation_1-auc:0.73001
[1325]	validation_0-auc:0.83750	validation_1-auc:0.73002
[1326]	validation_0-auc:0.83762

[1453]	validation_0-auc:0.84238	validation_1-auc:0.73003
[1454]	validation_0-auc:0.84247	validation_1-auc:0.73003
[1455]	validation_0-auc:0.84245	validation_1-auc:0.73001
[1456]	validation_0-auc:0.84253	validation_1-auc:0.72996
[1457]	validation_0-auc:0.84262	validation_1-auc:0.72997
[1458]	validation_0-auc:0.84266	validation_1-auc:0.72998
[1459]	validation_0-auc:0.84268	validation_1-auc:0.72997
[1460]	validation_0-auc:0.84269	validation_1-auc:0.72997
[1461]	validation_0-auc:0.84278	validation_1-auc:0.73011
[1462]	validation_0-auc:0.84279	validation_1-auc:0.73011
[1463]	validation_0-auc:0.84280	validation_1-auc:0.73018
[1464]	validation_0-auc:0.84282	validation_1-auc:0.73021
[1465]	validation_0-auc:0.84295	validation_1-auc:0.73026
[1466]	validation_0-auc:0.84296	validation_1-auc:0.73026
[1467]	validation_0-auc:0.84297	validation_1-auc:0.73019
[1468]	validation_0-auc:0.84318	validation_1-auc:0.73023
[1469]	validation_0-auc:0.84321	validation_1-auc:0.73024
[1470]	validation_0-auc:0.84335

[1597]	validation_0-auc:0.85022	validation_1-auc:0.73018
[1598]	validation_0-auc:0.85025	validation_1-auc:0.73025
[1599]	validation_0-auc:0.85030	validation_1-auc:0.73026
[1600]	validation_0-auc:0.85031	validation_1-auc:0.73025
[1601]	validation_0-auc:0.85032	validation_1-auc:0.73025
[1602]	validation_0-auc:0.85033	validation_1-auc:0.73025
[1603]	validation_0-auc:0.85034	validation_1-auc:0.73025
[1604]	validation_0-auc:0.85031	validation_1-auc:0.73020
[1605]	validation_0-auc:0.85032	validation_1-auc:0.73020
[1606]	validation_0-auc:0.85039	validation_1-auc:0.73021
[1607]	validation_0-auc:0.85041	validation_1-auc:0.73021
[1608]	validation_0-auc:0.85045	validation_1-auc:0.73019
[1609]	validation_0-auc:0.85046	validation_1-auc:0.73018
[1610]	validation_0-auc:0.85047	validation_1-auc:0.73018
[1611]	validation_0-auc:0.85051	validation_1-auc:0.73020
[1612]	validation_0-auc:0.85056	validation_1-auc:0.73014
[1613]	validation_0-auc:0.85072	validation_1-auc:0.73014
[1614]	validation_0-auc:0.85074

[1741]	validation_0-auc:0.85661	validation_1-auc:0.73019
[1742]	validation_0-auc:0.85670	validation_1-auc:0.73020
[1743]	validation_0-auc:0.85673	validation_1-auc:0.73021
[1744]	validation_0-auc:0.85675	validation_1-auc:0.73020
[1745]	validation_0-auc:0.85677	validation_1-auc:0.73033
[1746]	validation_0-auc:0.85682	validation_1-auc:0.73032
[1747]	validation_0-auc:0.85683	validation_1-auc:0.73032
[1748]	validation_0-auc:0.85684	validation_1-auc:0.73032
[1749]	validation_0-auc:0.85688	validation_1-auc:0.73018
[1750]	validation_0-auc:0.85692	validation_1-auc:0.73021
[1751]	validation_0-auc:0.85701	validation_1-auc:0.73023
[1752]	validation_0-auc:0.85702	validation_1-auc:0.73023
[1753]	validation_0-auc:0.85703	validation_1-auc:0.73016
[1754]	validation_0-auc:0.85704	validation_1-auc:0.73028
[1755]	validation_0-auc:0.85706	validation_1-auc:0.73022
[1756]	validation_0-auc:0.85707	validation_1-auc:0.73028
[1757]	validation_0-auc:0.85708	validation_1-auc:0.73027
[1758]	validation_0-auc:0.85709

[1885]	validation_0-auc:0.86241	validation_1-auc:0.73068
[1886]	validation_0-auc:0.86244	validation_1-auc:0.73068
[1887]	validation_0-auc:0.86253	validation_1-auc:0.73068
[1888]	validation_0-auc:0.86254	validation_1-auc:0.73070
[1889]	validation_0-auc:0.86264	validation_1-auc:0.73069
[1890]	validation_0-auc:0.86269	validation_1-auc:0.73069
[1891]	validation_0-auc:0.86269	validation_1-auc:0.73069
[1892]	validation_0-auc:0.86270	validation_1-auc:0.73069
[1893]	validation_0-auc:0.86275	validation_1-auc:0.73067
[1894]	validation_0-auc:0.86280	validation_1-auc:0.73069
[1895]	validation_0-auc:0.86286	validation_1-auc:0.73067
[1896]	validation_0-auc:0.86289	validation_1-auc:0.73067
[1897]	validation_0-auc:0.86290	validation_1-auc:0.73067
[1898]	validation_0-auc:0.86291	validation_1-auc:0.73066
[1899]	validation_0-auc:0.86291	validation_1-auc:0.73065
[1900]	validation_0-auc:0.86300	validation_1-auc:0.73057
[1901]	validation_0-auc:0.86309	validation_1-auc:0.73058
[1902]	validation_0-auc:0.86313

In [32]:
score = roc_auc_score(y_train, oof_preds_train_xgb)
score

0.866900181869759

In [17]:
score = roc_auc_score(y_valid, oof_preds_xgb)
score

0.730609719503322

In [18]:
np.corrcoef(oof_preds_train_xgb,oof_preds_train_lgbm)

array([[1.        , 0.89093009],
       [0.89093009, 1.        ]])

In [19]:
np.corrcoef(oof_preds_xgb,oof_preds_lgbm)

array([[1.       , 0.9294215],
       [0.9294215, 1.       ]])

Прогнозы хорошо коррелируют между собой

#### 2 Усреднить прогнозы с помощью арифмитического среднего, геометрического среднего и усреднить ранги, сделать выводы о качестве отдельных моделей и о качестве комбинации.

In [40]:
oof=pd.DataFrame(oof_preds_lgbm,columns=['lgbm'])
oof['xgb']=oof_preds_xgb

oof_train=pd.DataFrame(oof_preds_train_lgbm,columns=['lgbm'])
oof_train['xgb']=oof_preds_train_xgb

In [41]:
oof['mean_lgb_xgb']=oof[['lgbm','xgb']].mean(axis=1)
oof_train['mean_lgb_xgb']=oof_train[['lgbm','xgb']].mean(axis=1)

In [42]:
from scipy.stats.mstats import gmean

oof['gmean'] = gmean(oof.iloc[:,:2],axis=1)
oof_train['gmean'] = gmean(oof_train.iloc[:,:2],axis=1)

In [45]:
score = roc_auc_score(y_valid, oof['mean_lgb_xgb'])
score

0.7302519338474858

In [46]:
score_train = roc_auc_score(y_train, oof_train['mean_lgb_xgb'])
score_train

0.8377473650688203

In [47]:
score_train = roc_auc_score(y_train, oof_train['gmean'])
score_train

0.8326192465747327

In [48]:
score = roc_auc_score(y_valid, oof['gmean'])
score

0.7305674040626768

Качество модели улучшилась в геометрическом усреднении

#### 3.-4 Обучить CatBoost, получить OOF прогнозы и выполнить задание 1-2 для трех моделей.

In [26]:
x_train, x_valid = train_test_split(
      train, train_size=0.7, random_state=27
    )
y_train, y_valid = train_test_split(
        target, train_size=0.7, random_state=27
    )

cb_params = {
    "n_estimators": 2000,
    "learning_rate": 0.01,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
 #   "task_type": "CPU",
    "max_bin": 20,
    "verbose": 1000,
    "max_depth": 6,
    "l2_leaf_reg": 10,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
   
}
model_catboost = cb.CatBoostClassifier(**cb_params)
       
model_catboost.fit(x_train[numerical], y_train,eval_set=[(x_train[numerical], y_train), (x_valid[numerical], y_valid)])
oof_preds_catboost= model_catboost.predict_proba(x_valid[numerical])[:, 1]
oof_preds_train_catboost=model_catboost.predict_proba(x_train[numerical])[:, 1]

0:	test: 0.6694943	test1: 0.6772345	best: 0.6772345 (0)	total: 212ms	remaining: 7m 3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7231053833
bestIteration = 851

Shrink model to first 852 iterations.


In [56]:
oof['catboost']=oof_preds_catboost
oof_train['catboost']=oof_preds_train_catboost

In [57]:
score_train = roc_auc_score(y_train, oof_train['catboost'])
score_train

0.7345004236193198

In [58]:
score = roc_auc_score(y_valid, oof['catboost'])
score

0.7231053832577872

In [59]:
oof['mean_lgb_xgb_catboost']=oof[['lgbm','xgb','catboost']].mean(axis=1)
oof_train['mean_lgb_xgb_catboost']=oof_train[['lgbm','xgb','catboost']].mean(axis=1)

In [60]:
oof[['lgbm','xgb','catboost']].corr()

,lgbm,xgb,catboost
lgbm,1.000000,0.929421,0.960056
xgb,0.929421,1.000000,0.906194
catboost,0.960056,0.906194,1.000000


In [61]:
score_train = roc_auc_score(y_train, oof_train['mean_lgb_xgb_catboost'])
score_train

0.8159445925841016

In [62]:
score = roc_auc_score(y_valid, oof['mean_lgb_xgb_catboost'])
score

0.7299207721922989

In [78]:
oof['gmean'] = gmean(oof.iloc[:,[0,1,4]],axis=1)
oof_train['gmean'] = gmean(oof_train.iloc[:,[0,1,4]],axis=1)

In [79]:
score_train = roc_auc_score(y_train, oof_train['gmean'])
score_train

0.8058583609121629

In [80]:
score = roc_auc_score(y_valid, oof['gmean'])
score

0.7300224541933976

На valid и train сократился разрыв после усреднения по трем моделям.(прогноз стал более стабильный) качество возросло. Конечно сильного прироста ожидать не приходится т.к. используем однотипные алгоритмы.  За счет усреднения корректируем ошибку единичного алгоритма. 

#### 5. (опция) Объединить OOF-прогнозы для трех моделей и обучить алгоритм Логистической регрессии (и любой другой, на ваше усмотрение). Сделать выводы о достигаемом качестве, сравнить достигаемое качество с качеством отдельных моделей и моделей, полученных в п.2 и п.4.

In [109]:
column=['lgbm','xgb','catboost']
oof_lr=oof[column]
oof_lr_train=oof_train[column]

In [110]:
from sklearn.linear_model import LogisticRegression as lr

In [111]:
lr_model=lr(l1_ratio=0,random_state=27).fit(oof_lr_train,y_train)

In [112]:
oof_lr['lr_model']=lr_model.predict_proba(oof_lr)[:,1]

In [113]:
score = roc_auc_score(y_valid, oof_lr['lr_model'])
score

0.6740862434125504

Качество на тестовых данных существенно упало.Учитывая что применяем однотипные алгоритмы и OOF прогнозы сильно кореллируют между собой качество хуже чем при применении обычного усреднения 